In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset


In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=3 * 28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 20)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [ ]:
hyperparams = {
    "num_epochs": 64,
    "batch_size": 256,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.7}, # 1.0 -> no edges, 0.0 -> all edges
    "choose_thresholds_del": {"fc0": 0.1}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 0.005,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 1e-2,
    "delete_after": 2,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, batch_size: 256, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.7}, choose_thresholds_del: {'fc0': 0.1}, threshold: 0.005, min_delta_epoch_replace: 8, window_size: 5, lr: 0.001, delete_after: 2, task_type: classification, fully_connected: False, max_to_replace: 900"

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

train_x = np.load("data/AddNIST/train_x.npy")
train_y = np.load("data/AddNIST/train_y.npy")
valid_x = np.load("data/AddNIST/valid_x.npy")
valid_y = np.load("data/AddNIST/valid_y.npy")

train_tensor_x = torch.tensor(train_x).float().view(-1, 3 * 28 * 28)
train_tensor_y = torch.tensor(train_y).long()

valid_tensor_x = torch.tensor(valid_x).float().view(-1, 3 * 28 * 28)
valid_tensor_y = torch.tensor(valid_y).long()

train_dataset = TensorDataset(train_tensor_x, train_tensor_y)
valid_dataset = TensorDataset(valid_tensor_x, valid_tensor_y)

train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=hyperparams['batch_size'], shuffle=False)

In [7]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0], device=device)

In [8]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
wandb.finish()
run = wandb.init(
    project="self-expanding-nets-AddNIST",
    name=f"trash",
    config=hyperparams
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hyperparams['lr'])
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, optimizer, hyperparams, device)

100%|██████████| 176/176 [00:06<00:00, 29.00it/s]


Epoch 1/64, Train Loss: 3.1464, Val Loss: 3.1457, Val Accuracy: 0.0569


100%|██████████| 176/176 [00:05<00:00, 31.33it/s]


Epoch 2/64, Train Loss: 3.1464, Val Loss: 3.1457, Val Accuracy: 0.0569


100%|██████████| 176/176 [00:05<00:00, 31.67it/s]


...

Epoch 62/64, Train Loss: 1.8357, Val Loss: 2.4660, Val Accuracy: 0.2203


100%|██████████| 176/176 [00:16<00:00, 10.82it/s]


Epoch 63/64, Train Loss: 1.8407, Val Loss: 2.4452, Val Accuracy: 0.2273


100%|██████████| 176/176 [00:15<00:00, 11.65it/s]


Epoch 64/64, Train Loss: 1.8303, Val Loss: 2.4642, Val Accuracy: 0.2206
